<a href="https://www.kaggle.com/code/kparthiban007/pytorch-cnn?scriptVersionId=222641755" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="background-color:#ca5f5c; color:#ffff; padding:15px; text-align:center; border-radius:15px; font-size:25px;">PyTorch-CNN</div>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim  as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
train_path = '/kaggle/input/cats-and-dogs/train'
test_path = '/kaggle/input/cats-and-dogs/val'

In [3]:
image_size = 224  
batch_size = 32   

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [4]:
train_dataset = datasets.ImageFolder(root = train_path ,transform = transform)
train_loader = DataLoader(train_dataset,batch_size = batch_size ,shuffle = True)


test_dataset = datasets.ImageFolder(root = test_path,transform = transform)
test_loader = DataLoader(test_dataset , batch_size = batch_size , shuffle = False)

In [5]:
classes = train_dataset.classes
print(" Class Names : ",classes)

 Class Names :  ['cat', 'dog']


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3 ,out_channels = 64,kernel_size =3 ,padding =1)
        self.conv2 = nn.Conv2d(in_channels = 64,out_channels = 128,kernel_size = 3,padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 128,out_channels =256 ,kernel_size = 3,padding =1)
        
        self.pool = nn.MaxPool2d(kernel_size = 2,stride = 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        self.flattened_size = 128 * (image_size // 8) * (image_size // 8)

        self.fc1 = nn.Linear(256*28*28,256)
        self.fc2 = nn.Linear(256,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1) 
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))  
        return x
        
        

In [7]:
device = torch.device("cuda") if torch.cuda.is_available else "cpu"
print(device)
model = CNN().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr =0.0021)


cuda


In [8]:
def train(model,train_loader,criterion,optimizer,epochs =10):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for images , label in train_loader:
            images , labels = images.to(device) , label.to(device).float().unsqueeze(1)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
        
       
        
        print(f"Epoch {epoch+1}/{epochs} , Loss : {running_loss/len(train_loader):.4f}")


train(model,train_loader,criterion, optimizer,epochs =10)

Epoch 1/10 , Loss : 32.4234
Epoch 2/10 , Loss : 35.1243
Epoch 3/10 , Loss : 34.6491
Epoch 4/10 , Loss : 34.1740
Epoch 5/10 , Loss : 35.3618
Epoch 6/10 , Loss : 33.9364
Epoch 7/10 , Loss : 34.4115
Epoch 8/10 , Loss : 34.4115
Epoch 9/10 , Loss : 34.6491
Epoch 10/10 , Loss : 34.6491


In [9]:
def evaluate(model, test_loader, criterion):
    model.eval()
    
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            predictions = (outputs > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")
    
    return avg_loss, accuracy  

avg_loss , accuracy = evaluate(model,test_loader, criterion)

Validation Loss: 25.0000, Validation Accuracy: 65.71%


In [10]:
avg_loss , accuracy 

(25.0, 65.71428571428571)

<div style="background-color:#ca5f5c; color:#ffff; padding:15px; text-align:center; border-radius:15px; font-size:25px;">CNN - ResNet</div>

In [11]:
import torchvision.models as models

model = models.resnet50(pretrained=True)
model.eval()  

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:

num_classes = 2  
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [13]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:  
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete.")


Epoch [1/5], Loss: 0.3687
Epoch [2/5], Loss: 0.3840
Epoch [3/5], Loss: 0.3084
Epoch [4/5], Loss: 0.1534
Epoch [5/5], Loss: 0.0879
Training complete.


In [15]:
import torch

def evaluate(model, test_loader, criterion, device):
    model.eval()  
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy


In [16]:
test_loss, test_accuracy = evaluate(model, test_loader, criterion,device)

Test Loss: 0.3485, Accuracy: 90.00%


<div style="background-color:#ca5f5c; color:#ffff; padding:15px; text-align:center; border-radius:15px; font-size:25px;">CNN - Xception</div>

In [17]:
import timm

base_model = timm.create_model('xception', pretrained=True, num_classes=0)  

class XceptionBinaryClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(XceptionBinaryClassifier, self).__init__()
        self.xception = timm.create_model('xception', pretrained=True, num_classes=0)  
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),  
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)  
        )

    def forward(self, x):
        x = self.xception(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

num_classes = 2  
model = XceptionBinaryClassifier(num_classes=num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.0001)


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


In [18]:
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

    print("Training complete.")


In [19]:
def evaluate(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    avg_loss = total_loss / len(test_loader)
    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy


In [20]:
train(model, train_loader, criterion, optimizer, epochs=5)

avg_loss, accuracy = evaluate(model, test_loader, criterion)


Epoch [1/5], Loss: 0.5976
Epoch [2/5], Loss: 0.2838
Epoch [3/5], Loss: 0.1000
Epoch [4/5], Loss: 0.0251
Epoch [5/5], Loss: 0.0085
Training complete.
Test Loss: 0.0238, Accuracy: 98.57%


In [21]:
avg_loss, accuracy

(0.02377270519112547, 98.57142857142857)